## Installs and Imports
Install and import PyTorch along with a few helper libraries

## import 

In [ ]:
# import torch
# torch.cuda.is_available()

In [ ]:
# import sys
# sys.path.insert(0, r"/home/sanjeev_kumar/work/polaris-ai-development-faster_rcnn_end_to_end")

In [8]:
# basic python and ML Libraries
import os
import datetime
import random
import numpy as np
import pandas as pd
# import pandas as pd
import seaborn as sns
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# # helper libraries
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from pycocotools.coco import COCO
import albumentations as A
from collections import defaultdict
from PIL import Image, ImageDraw,ImageFont 
import torchvision.transforms as transforms
print("torch.cuda.is_available() : ",torch.cuda.is_available())
from PIL import Image

ModuleNotFoundError: No module named 'cv2'

## The Dataset 

In [9]:
"""
CustomDataset : create class that 
and return image , target dict 

"""


class CustomDataset(torch.utils.data.Dataset):
    """
    This is class create
    """
    def __init__(self, img_dir, annot_path, transforms=False, normalize=True):
        """
        Args:
            img_dir (path): image directory path
            annot_path (json_path): coco json path
            transforms (bool): if true, albumentation transfomation is applied on image
            normalize (boll): if true , ToTensor transforation is applied on the image which convert image pixel to [0,1] and
                                        return image in form (C,H, W)
        """
        self.transforms = transforms
        self.normalize = normalize
        self.img_dir = img_dir
        self.annot_path = annot_path
        self.coco = COCO(self.annot_path)
        self.img_ids = self.coco.getImgIds()

    #         print("self.img_ids : ",self.img_ids)

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_obj = self.coco.loadImgs(img_id)[0]
        image_name = img_obj["file_name"]
        image_path = os.path.join(self.img_dir, image_name)
        # image_path = image_path[:-4]+ image_path[-4:].upper() 
        image_path = image_path[:-4]+ image_path[-4:].lower() 
        # print(image_path)
        image = cv2.imread(image_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        anns_obj = self.coco.loadAnns(self.coco.getAnnIds(img_id))

        anno_data = defaultdict(list)
        for ann in anns_obj:
            box = ann['bbox']
            xmin = box[0]
            ymin = box[1]
            xmax = box[0] + box[2]
            ymax = box[1] + box[3]
            anno_data["boxes"].append([xmin, ymin, xmax, ymax])
            #             anno_data["boxes"].append(ann['bbox'])
            anno_data['area'].append(ann['area'])
            anno_data['iscrowd'].append(ann['iscrowd'])
            anno_data['label'].append(ann['category_id'])
            anno_data["cat_name"].append(self.get_cat_names(ann['category_id']))

        # applying the albumenation transormation here for augmentation.
        if self.transforms:
            transformed_img, transformed_bboxes, transformed_class_labels = self.transform_img(img=image,
                                                                                               bbox=anno_data['boxes'],
                                                                                               label=anno_data["label"])
            # replcing the original image,box,label with transformed
            image = transformed_img
            anno_data['boxes'] = transformed_bboxes
            anno_data["label"] = transformed_class_labels

        if self.normalize:  # converting image to normalize (pixel value 0 to 1) tensor (C,H,W) -> float32
            transform = transforms.Compose([transforms.ToTensor()])
            image = transform(image)

        target = {
            'boxes': torch.as_tensor(anno_data['boxes'], dtype=torch.int64),
            'labels': torch.as_tensor(anno_data["label"], dtype=torch.int64),
            'image_id': torch.tensor([img_id]),  # pylint: disable=not-callable (false alarm)
            'area': torch.as_tensor(anno_data['area'], dtype=torch.float32),
            'iscrowd': torch.as_tensor(anno_data['iscrowd'], dtype=torch.int64),
#             'cat_name': anno_data["cat_name"],
        }

        return image, target

    def transform_img(self, img, bbox, label):

        """
        apply albumentation transoformation on image,bbox,label
        Args:
            self ():
            img (numpy.ndarray): image to transform
            bbox (list): bounding box list
            label (list:str): label of each bounding box

        Returns:
            tuple of transformed_image,transformed_bboxes,transformed_labels
        """
        # this is link alubumentation
        # https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/    -- for bounding box understanding
        # https://albumentations.ai/docs/ - doc file
        # https://github.com/albumentations-team/albumentations  -- list of all other transform used in Albumentation

        transform = A.Compose([
            A.Resize(*img_size,),
            A.HorizontalFlip(),
            A.RandomBrightnessContrast(brightness_limit = 0.3, contrast_limit = 0.2), #p=0.3
            # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.5, rotate_limit=55, p=0.3),
            # A.SafeRotate(limit=45, p=1)
        # ], bbox_params=A.BboxParams(format='pascal_voc', min_area=50, min_visibility=0.3, label_fields=['class_labels']))
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

        transformed = transform(image=img, bboxes=bbox, class_labels=label)
        transformed_img = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_labels = transformed['class_labels']

        if not transformed_bboxes:
            transform = A.Compose([
                                    A.Resize(*img_size,),
                                    A.HorizontalFlip(p=0.5),
                                    A.RandomBrightnessContrast(brightness_limit = 0.3, contrast_limit = 0.2, p=0.3), #p=0.3
                                ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))
            transformed = transform(image=img, bboxes=bbox, class_labels=label)
            transformed_img = transformed['image']
            transformed_bboxes = transformed['bboxes']
            transformed_labels = transformed['class_labels']
            return transformed_img, transformed_bboxes, transformed_labels

        #     transformed_bboxes.append([0.0001, 0.0001, 0.0001, 0.0001])
        #     transformed_labels.append(0)

        return transformed_img, transformed_bboxes, transformed_labels

    def get_cat_names(self, cat_ids=[]):
        return [data['name'] for data in self.coco.loadCats(cat_ids)][0]

NameError: name 'torch' is not defined

## Visualization

Let's make some a helper function to view our data

In [10]:
def draw_bounding_box(img, pred_dict={},text_color="red",text_font_size = 50,bbox_color = "white",bbox_thickness = 5):  # drow bounding box on the image
    id_2_class = {0: '_background_', 1: 'tire_marks', 2: 'trackout', 3: 'unclear'}
    if isinstance(img,np.ndarray):
        image =Image.fromarray(img.copy())
    else:
        image =img.copy()
    font = ImageFont.truetype(r"/home/sanjeev_kumar/work/trackouts/SiemensSans_Global_Roman.ttf", size=text_font_size)
    bbox_list = pred_dict["boxes"].cuda().cpu().numpy()
    labels = pred_dict["labels"].cuda().cpu().numpy()
    scores = pred_dict["scores"].cuda().cpu().numpy()
    for i,bbox in enumerate(bbox_list):
        xmin, ymin, xmax, ymax = bbox
        start_point = (xmin, ymax)
        end_point = (xmax, ymin)
        draw = ImageDraw.Draw(image)
        draw.rectangle((start_point, end_point), outline =bbox_color,width=bbox_thickness)
        text_point =(xmin, ymin - 50)
        draw.text(text_point, id_2_class[labels[i]] , fill=text_color,font=font)
    return image




# Function to visualize bounding boxes in the image
def plot_img_bbox(img, target):
  # plot the image and bboxes
  # Bounding boxes are defined as follows: x-min y-min width height
#     img = torch.tensor(img.numpy()*255,dtype=torch.uint8)
    if isinstance(img,torch.Tensor):
        img = torchtrans.ToPILImage()(img).convert('RGB')
        
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    boxes  = target['boxes']
    if isinstance(boxes,torch.Tensor):
        boxes = boxes.cpu()
    for box in (boxes):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle(
          (x, y),
          width, height,
          linewidth = 2,
          edgecolor = 'r',
          facecolor = 'none'
        )
        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()
    

## collate function 

In [11]:
def my_collate(batch):
    batch = filter (lambda x:x is not None, batch)
    return utils.collate_fn(batch)


# def my_collate(batch, dataset):
#     len_batch = len(batch)
#     batch = list(filter(lambda x: x is not None, batch))

#     if len_batch > len(batch):                
#         db_len = len(dataset)
#         diff = len_batch - len(batch)
#         while diff != 0:
#             a = dataset[np.random.randint(0, db_len)]
#             if a is None:                
#                 continue
#             batch.append(a)# Dataloaders

# Make a loader for feeding our data into the neural network
#             diff -= 1

#     return torch.utils.data.dataloader.default_collate(batch)

# Dataset

Make a loader for feeding our data into the neural network

### Constants

In [12]:
img_size = [1024,1024]
# img_size = [512,512] #[1024,1024]
num_worker = 0
batch_size = 8
num_epochs = 100
# train on gpu if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 4 # one class (class 0) is dedicated to the "background"


NameError: name 'torch' is not defined

### working dir

In [13]:
# import os
# print("before : ",os.getcwd())
# os.chdir(r"/home/sanjeev_kumar/work")
# print("after : ",os.getcwd())

### path and dataset

In [14]:
pwd

'D:\\work\\downloaded_zip\\temp'

In [15]:
import os
os.path.exists("trackouts_patch/Images")

False

In [16]:
usecase_name = "trackouts_patch" #coco_fridge_data  fpc
IMG_DIR = f'./{usecase_name}/Images/'
TRAIN_ANNOT_PATH = f"./{usecase_name}/train.json"
# validation images and test.json files directory/path
VALID_ANNOT_PATH = f"./{usecase_name}/test.json"

# TRAIN_ANNOT_PATH = r"/home/sanjeev_kumar/work/trakouts/train.json"
# VALID_ANNOT_PATH = r"/home/sanjeev_kumar/work/trakouts/test.json"

# train_dataset = CustomDataset(IMG_DIR,TRAIN_ANNOT_PATH,transforms=True, normalize=True)
# img, target = train_dataset[6]
# print('Image shape:', img.shape)
# print('Label example:', target)
test_dataset = CustomDataset(IMG_DIR,VALID_ANNOT_PATH,transforms=True, normalize=True)
img, target = test_dataset[8]
print('Image shape:', img.shape)
print('Label example:', target)

NameError: name 'CustomDataset' is not defined

In [17]:
# plotting the image with bboxes. Feel free to change the index
img, target = test_dataset[3]
plot_img_bbox(img, target)

NameError: name 'test_dataset' is not defined

# Dataloader

In [18]:
# # use our dataset and defined transformations

dataset = CustomDataset(IMG_DIR,TRAIN_ANNOT_PATH,transforms=True, normalize=True)
dataset_test =  CustomDataset(IMG_DIR,VALID_ANNOT_PATH,transforms=True, normalize=True)

def my_collate(batch):
    batch = filter (lambda x:x is not None, batch)
    return utils.collate_fn(batch)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
  dataset,
  batch_size=batch_size,
  shuffle=True,
  num_workers=num_worker,
  # collate_fn=my_collate,
    collate_fn = utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
  dataset_test,
  batch_size=batch_size,
  shuffle=False,
  num_workers=num_worker,
  # collate_fn=my_collate,
  collate_fn =utils.collate_fn
)

NameError: name 'CustomDataset' is not defined

# Modeling

## Pre-trained Model

In [19]:
def get_object_detection_model(num_classes,pred=False):
    # load a model pre-trained pre-trained on COCO
    if pred:
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
    else:
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    return model

## eval function

In [20]:
from collections import OrderedDict

def eval_forward(model, images, targets):
    """type: (List[Tensor], Optional[List[Dict[str, Tensor]]]) -> Tuple[Dict[str, Tensor], List[Dict[str, Tensor]]]"""

    """
    Args:
        images (list[Tensor]): images to be processed
        targets (list[Dict[str, Tensor]]): ground-truth boxes present in the image (optional)
    Returns:
        result (list[BoxList] or dict[Tensor]): the output from the model.
            It returns list[BoxList] contains additional fields
            like `scores`, `labels` and `mask` (for Mask R-CNN models).
    """
    model.eval()

    original_image_sizes: List[Tuple[int, int]] = []
    for img in images:
        val = img.shape[-2:]
        assert len(val) == 2
        original_image_sizes.append((val[0], val[1]))

    images, targets = model.transform(images, targets)

    # Check for degenerate boxes
    # TODO: Move this to a function
    if targets is not None:
        for target_idx, target in enumerate(targets):
            boxes = target["boxes"]
            degenerate_boxes = boxes[:, 2:] <= boxes[:, :2]
            if degenerate_boxes.any():
                # print the first degenerate box
                bb_idx = torch.where(degenerate_boxes.any(dim=1))[0][0]
                degen_bb: List[float] = boxes[bb_idx].tolist()
                raise ValueError(
                    "All bounding boxes should have positive height and width."
                    f" Found invalid box {degen_bb} for target at index {target_idx}."
                )

    features = model.backbone(images.tensors)
    if isinstance(features, torch.Tensor):
        features = OrderedDict([("0", features)])
    model.rpn.training=True
    #model.roi_heads.training=True


    #####proposals, proposal_losses = model.rpn(images, features, targets)
    features_rpn = list(features.values())
    objectness, pred_bbox_deltas = model.rpn.head(features_rpn)
    anchors = model.rpn.anchor_generator(images, features_rpn)

    num_images = len(anchors)
    num_anchors_per_level_shape_tensors = [o[0].shape for o in objectness]
    num_anchors_per_level = [s[0] * s[1] * s[2] for s in num_anchors_per_level_shape_tensors]
    objectness, pred_bbox_deltas = concat_box_prediction_layers(objectness, pred_bbox_deltas)
    # apply pred_bbox_deltas to anchors to obtain the decoded proposals
    # note that we detach the deltas because Faster R-CNN do not backprop through
    # the proposals
    proposals = model.rpn.box_coder.decode(pred_bbox_deltas.detach(), anchors)
    proposals = proposals.view(num_images, -1, 4)
    proposals, scores = model.rpn.filter_proposals(proposals, objectness, images.image_sizes, num_anchors_per_level)

    proposal_losses = {}
    assert targets is not None
    labels, matched_gt_boxes = model.rpn.assign_targets_to_anchors(anchors, targets)
    regression_targets = model.rpn.box_coder.encode(matched_gt_boxes, anchors)
    loss_objectness, loss_rpn_box_reg = model.rpn.compute_loss(
        objectness, pred_bbox_deltas, labels, regression_targets
    )
    proposal_losses = {
        "loss_objectness": loss_objectness,
        "loss_rpn_box_reg": loss_rpn_box_reg,
    }

    #####detections, detector_losses = model.roi_heads(features, proposals, images.image_sizes, targets)
    image_shapes = images.image_sizes
    proposals, matched_idxs, labels, regression_targets = model.roi_heads.select_training_samples(proposals, targets)
    box_features = model.roi_heads.box_roi_pool(features, proposals, image_shapes)
    box_features = model.roi_heads.box_head(box_features)
    class_logits, box_regression = model.roi_heads.box_predictor(box_features)

    result: List[Dict[str, torch.Tensor]] = []
    detector_losses = {}
    loss_classifier, loss_box_reg = fastrcnn_loss(class_logits, box_regression, labels, regression_targets)
    detector_losses = {"loss_classifier": loss_classifier, "loss_box_reg": loss_box_reg}
    boxes, scores, labels = model.roi_heads.postprocess_detections(class_logits, box_regression, proposals, image_shapes)
    num_images = len(boxes)
    for i in range(num_images):
        result.append(
            {
                "boxes": boxes[i],
                "labels": labels[i],
                "scores": scores[i],
            }
        )
    detections = result
    detections = model.transform.postprocess(detections, images.image_sizes, original_image_sizes)  # type: ignore[operator]
    model.rpn.training=False
    model.roi_heads.training=False
    losses = {}
    losses.update(detector_losses)
    losses.update(proposal_losses)
    return losses, detections


from typing import Dict, List, Optional, Tuple
from torch import nn, Tensor

def concat_box_prediction_layers(box_cls: List[Tensor], box_regression: List[Tensor]) -> Tuple[Tensor, Tensor]:
    box_cls_flattened = []
    box_regression_flattened = []
    # for each feature level, permute the outputs to make them be in the
    # same format as the labels. Note that the labels are computed for
    # all feature levels concatenated, so we keep the same representation
    # for the objectness and the box_regression
    for box_cls_per_level, box_regression_per_level in zip(box_cls, box_regression):
        N, AxC, H, W = box_cls_per_level.shape
        Ax4 = box_regression_per_level.shape[1]
        A = Ax4 // 4
        C = AxC // A
        box_cls_per_level = permute_and_flatten(box_cls_per_level, N, A, C, H, W)
        box_cls_flattened.append(box_cls_per_level)

        box_regression_per_level = permute_and_flatten(box_regression_per_level, N, A, 4, H, W)
        box_regression_flattened.append(box_regression_per_level)
    # concatenate on the first dimension (representing the feature levels), to
    # take into account the way the labels were generated (with all feature maps
    # being concatenated as well)
    box_cls = torch.cat(box_cls_flattened, dim=1).flatten(0, -2)
    box_regression = torch.cat(box_regression_flattened, dim=1).reshape(-1, 4)
    return box_cls, box_regression

def permute_and_flatten(layer: Tensor, N: int, A: int, C: int, H: int, W: int) -> Tensor:
    layer = layer.view(N, -1, C, H, W)
    layer = layer.permute(0, 3, 4, 1, 2)
    layer = layer.reshape(N, -1, C)
    return layer

import torch.nn.functional as F

def fastrcnn_loss(class_logits, box_regression, labels, regression_targets):
    # type: (Tensor, Tensor, List[Tensor], List[Tensor]) -> Tuple[Tensor, Tensor]
    """
    Computes the loss for Faster R-CNN.
    Args:
        class_logits (Tensor)
        box_regression (Tensor)
        labels (list[BoxList])
        regression_targets (Tensor)
    Returns:
        classification_loss (Tensor)
        box_loss (Tensor)
    """

    labels = torch.cat(labels, dim=0)
    regression_targets = torch.cat(regression_targets, dim=0)

    classification_loss = F.cross_entropy(class_logits, labels)

    # get indices that correspond to the regression targets for
    # the corresponding ground truth labels, to be used with
    # advanced indexing
    sampled_pos_inds_subset = torch.where(labels > 0)[0]
    labels_pos = labels[sampled_pos_inds_subset]
    N, num_classes = class_logits.shape
    box_regression = box_regression.reshape(N, box_regression.size(-1) // 4, 4)

    box_loss = F.smooth_l1_loss(
        box_regression[sampled_pos_inds_subset, labels_pos],
        regression_targets[sampled_pos_inds_subset],
        beta=1 / 9,
        reduction="sum",
    )
    box_loss = box_loss / labels.numel()

    return classification_loss, box_loss

ModuleNotFoundError: No module named 'torch'

## evaluate_loss

In [21]:
def evaluate_loss(model, data_loader, device):
    val_loss = 0
    with torch.no_grad():
      for images, targets in data_loader:
          images = list(image.to(device) for image in images)
          targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

          losses_dict, detections = eval_forward(model, images, targets)
          losses = sum(loss for loss in losses_dict.values())
          
          val_loss += losses
          
    validation_loss = val_loss/ len(data_loader)    
    return validation_loss

## apply_nms

In [22]:
# the function takes the original prediction and the iou threshold.
def apply_nms(orig_prediction, iou_thresh=0.5):
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)

    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    return final_prediction

# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')

## EarlyStopping

In [23]:
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if torch.isnan(metrics):
            print("Metric reached nan, stopping training!!")
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            print("Training stopped since metric did not improve!!")
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

# Training

Let's prepare the model for training

In [24]:
print("num_classes : ",num_classes)
# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.00001, momentum=0.09, weight_decay=0.0005)
# optimizer = torch.optim.Adam(params, lr=0.005, weight_decay=0.0005)
optimizer = torch.optim.Adam(params, lr=0.0001)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(
  optimizer,
  step_size=3,
  gamma=0.1
)

NameError: name 'num_classes' is not defined

In [25]:
print("training for epochs = ",num_epochs)
print("batch_size : ",batch_size)

training for epochs =  100
batch_size :  8


In [26]:
es = EarlyStopping(patience=50)

train_loss_list =[]
val_loss_list =[]
mAP_score_list =[]

for epoch in range(num_epochs):
        train_loss =  train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        train_loss_list.append(train_loss)
        print('train_loss_list',train_loss_list)
        # mAP_score_list.append(mAP_score)
        # print('map score list',mAP_score_list)
        lr_scheduler.step()
        validation_loss  = evaluate_loss(model, data_loader_test, device=device)
        print(f"Validation loss ------------------> {validation_loss}")
        val_loss_list.append(validation_loss.cpu().item())
        # writer.add_scalar('validation loss',validation_loss,epoch)
        if es.step(torch.tensor([validation_loss])):
                break
        
        # evaluate on the test dataset
        mAP = evaluate(model, data_loader_test, device=device)
        mAP_score_list.append(mAP)
        print('map score list',mAP_score_list)


NameError: name 'train_one_epoch' is not defined

In [27]:
es = EarlyStopping(patience=50)

train_loss_list =[]
val_loss_list =[]
mAP_score_list =[]

for epoch in range(num_epochs):
        train_loss =  train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        train_loss_list.append(train_loss)
        print('train_loss_list',train_loss_list)
        # mAP_score_list.append(mAP_score)
        # print('map score list',mAP_score_list)
        lr_scheduler.step()
        validation_loss  = evaluate_loss(model, data_loader_test, device=device)
        print(f"Validation loss ------------------> {validation_loss}")
        val_loss_list.append(validation_loss.cpu().item())
        # writer.add_scalar('validation loss',validation_loss,epoch)
        if es.step(torch.tensor([validation_loss])):
                break
        
        # evaluate on the test dataset
        mAP = evaluate(model, data_loader_test, device=device)
        mAP_score_list.append(mAP)
        print('map score list',mAP_score_list)


NameError: name 'train_one_epoch' is not defined

In [28]:

try: 
    data = {'train_loss': train_loss_list,'val_loss':val_loss_list, 'mAP':mAP_score_list}
    # data1 = {'mAP':mAP_score_list}
    loss_df = pd.DataFrame(data=data)
    # mAP_df = pd.DataFrame(data=data1)
    # mAP_df.to_csv('mAP/exp2.csv')
    loss_df.to_csv(f'./{usecase_name}/faster_rcnn_loss_1.csv')
except:
    data = {'train_loss': train_loss_list[:-1],'val_loss':val_loss_list[:-1], 'mAP':mAP_score_list}
    # data1 = {'mAP':mAP_score_list}
    loss_df = pd.DataFrame(data=data)
    # mAP_df = pd.DataFrame(data=data1)
    # mAP_df.to_csv('mAP/exp2.csv'2
    loss_df.to_csv(f'./{usecase_name}/faster_rcnn_loss_1.csv')
        

OSError: Cannot save file into a non-existent directory: 'trackouts_patch'

# save model

In [ ]:
model_name_weight = f"./{usecase_name}/fasterrcnn_"+str(num_epochs)+str("_")+f"{datetime.datetime.now():%Y_%m_%d}"+".pt"
model_name_shape = f"./{usecase_name}/fasterrcnn_shape_"+str(num_epochs)+str("_")+f"{datetime.datetime.now():%Y_%m_%d}"+".pt"
print("model_name_weight : ", model_name_weight)
print("model_name_shape : ", model_name_shape)
torch.save(model.state_dict(),model_name_weight)
torch.save(model, model_name_shape)

## training  plot

In [ ]:
loss_plot = sns.lineplot(data = loss_df)

plt.ylim(top=1) #ymax is your value
plt.ylim(bottom=0) #ymin is your value
plt.savefig( f'./{usecase_name}/loss_demo2.png')
plt.show(loss_plot)

# load model

In [ ]:
# model = torch.load('fasterrcnn_shape.pth')

# Model prediction 

In [29]:
# test_dataset = ConeImagesDataset(test_dir, 480, 480, transforms= get_transform(train=True))
idx=1
# pick one image from the test set
img, target = dataset_test[idx]
print(img.shape)
print(type(img))
# i =img
# put the model in evaluation mode
# model.cuda()
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
# print("type of prediction : ",type(prediction))
print("prediction : ",prediction)
print('MODEL OUTPUT\n')
nms_prediction = apply_nms(prediction, iou_thresh=0.01)

NameError: name 'dataset_test' is not defined

In [30]:
img = torch_to_pil(img)
bbox_img = draw_bounding_box(img,nms_prediction)
save_dir =r"/home/sanjeev_kumar/work/usecase/trackouts_patch/pred_img/"
save_path = save_dir +"pred_"+str(idx)+".JPG"
bbox_img.save(save_path)

# bbox_img

NameError: name 'img' is not defined

In [31]:
for i in range(30):
    
    idx=i
    img, target = dataset_test[idx]
    # model.cuda()
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])[0]
    nms_prediction = apply_nms(prediction, iou_thresh=0.01)

    img = torch_to_pil(img)
    bbox_img = draw_bounding_box(img,nms_prediction)
    save_dir =r"/home/sanjeev_kumar/work/usecase/trackouts_patch/pred_img/"
    save_path = save_dir +"pred_"+str(idx)+".JPG"
    bbox_img.save(save_path)
    # break

NameError: name 'dataset_test' is not defined

In [32]:
print(len(glob.glob("/home/sanjeev_kumar/work/usecase/trackouts_patch/pred_img/*")))

NameError: name 'glob' is not defined

In [33]:
img = torch_to_pil(img)
bbox_img = draw_bounding_box(img,nms_prediction)
# bbox_img.save(IMG_SAVE_PATH+name+ext
bbox_img

NameError: name 'img' is not defined

In [34]:
data_transforms = {
    # 'train': transforms.Compose([
    #     transforms.Resize((input_image_height, input_image_width)),
    #     transforms.ToTensor(),
    #     # transforms.Normalize(model_mean, model_std)
    # ]),
    'val': transforms.Compose([
        transforms.Resize(img_size[0]),
        transforms.ToTensor(),
        # transforms.Normalize(model_mean, model_std)
    ]),
}

NameError: name 'transforms' is not defined

In [35]:
'/home/sanjeev_kumar/work/trackouts/Images'

'/home/sanjeev_kumar/work/trackouts/Images'

In [36]:
from PIL import Image
import sys
# img_path = "./trackouts/Images/DJI_20211123144914_0002_W.jpg"
img_path = "/home/sanjeev_kumar/work/usecase/trackouts_patch/new_blind_img/DJI_20220928155042_0007_crop.JPG"


png_image = Image.open(img_path)
png_array = np.array(png_image)
png_image = Image.fromarray(png_array)
png_image = data_transforms['val'](png_image)
png_image = png_image.to(torch.float)
# png_image = torch.unsqueeze(png_image, 0)
png_image = png_image.to(device)




model.cuda()
model.eval()
with torch.no_grad():
    prediction = model([png_image])[0]
    
print('MODEL OUTPUT\n')
nms_prediction = apply_nms(prediction, iou_thresh=0.00)
plot_img_bbox(torch_to_pil(img), nms_prediction)
plot_img_bbox(png_image, nms_prediction)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sanjeev_kumar/work/usecase/trackouts_patch/new_blind_img/DJI_20220928155042_0007_crop.JPG'

In [ ]:
img = torch_to_pil(img.cuda().cpu())
bbox_img = draw_bounding_box(img,nms_prediction)
# bbox_img.save(IMG_SAVE_PATH+name+ext)
bbox_img

# Sahi prediction

In [37]:
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi import AutoDetectionModel
from IPython.display import Image

ModuleNotFoundError: No module named 'sahi'

In [38]:
def extract_sahi_pred(result_dict,image_size = (1024, 1024)):
    # print("len of the result : ",len(result_dict))
    # bbox_list,score_list,label_list,cat_name_list =[],[],[],[]
    sahi_pred_list=[]
    if len(result_dict)==1:
        temp={}
        # try:
        #     l = len(result_dict[0]['bbox'][0])
        #     islist= True
            
        # except :
        #     islist = False
        # if islist:
        #     temp["boxes"] =result_dict[0]['bbox']
        #     temp["labels"] =result_dict[0]['category_id']
        #     temp["scores"] =result_dict[0]['score']
              
        # else:
        #     temp["boxes"] =[result_dict[0]['bbox']]
        #     temp["labels"] = [result_dict[0]['category_id']]
        #     temp["scores"] =[result_dict[0]['score']]

        temp["boxes"] =[result_dict[0]['bbox']]
        temp["labels"] = [result_dict[0]['category_id']]
        temp["scores"] =[result_dict[0]['score']]
        sahi_pred_list.append(temp)
        return sahi_pred_list


    for item in result_dict:
        temp={}
        temp["boxes"] =item['bbox']
        temp["labels"] =item['category_id']
        temp["scores"] =item['score']
        sahi_pred_list.append(temp)


    return sahi_pred_list

# unseen data prediction

In [39]:
# /home/sanjeev_kumar/work/usecase/trackouts_patch/new_unseen_data/DJI_20221020112922_0049.JPG
UNSEEN_DATA_PATH =r"/home/sanjeev_kumar/work/usecase/trackouts_patch/new_blind_img/"
import glob
img_path_list = glob.glob(UNSEEN_DATA_PATH+'*.JPG')
print("img_path_list : ",len(img_path_list))
print(img_path_list[1])

img_path_list :  0


IndexError: list index out of range

In [40]:
def sahi_draw_bounding_box(img_path, pred_list=[],text_color="red",text_font_size = 50,bbox_color = "white",bbox_thickness = 5):  # drow bounding box on the image
    id_2_class = {0: '_background_', 1: 'tire_marks', 2: 'trackout', 3: 'unclear'}
    image = Image.open(img_path)
    font = ImageFont.truetype(r"/home/sanjeev_kumar/work/trackouts/SiemensSans_Global_Roman.ttf", size=text_font_size)
    for pred_dict in pred_list:
        bbox_list = pred_dict["boxes"]
        labels = pred_dict["labels"]
        scores = pred_dict["scores"]

        # xmin, ymin, xmax, ymax = bbox
        xmin, ymin, xmax, ymax = pred_dict["boxes"]
        xmin, ymin, xmax, ymax = xmin, ymin, xmin+xmax, ymin+ymax

        start_point = (xmin, ymax)
        end_point = (xmax, ymin)
        draw = ImageDraw.Draw(image)
        draw.rectangle((start_point, end_point), outline =bbox_color,width=bbox_thickness)
        text_point =(xmin, ymin - 50)
        draw.text(text_point, id_2_class[labels] , fill=text_color,font=font)

        # for i,bbox in enumerate(bbox_list):
        #     # xmin, ymin, xmax, ymax = bbox
        #     xmin, ymin, xmax, ymax = bbox
        #     xmin, ymin, xmax, ymax = xmin, ymin, xmin+xmax, ymin+ymax

        #     start_point = (xmin, ymax)
        #     end_point = (xmax, ymin)
        #     draw = ImageDraw.Draw(image)
        #     draw.rectangle((start_point, end_point), outline =bbox_color,width=bbox_thickness)
        #     text_point =(xmin, ymin - 50)
        #     draw.text(text_point, id_2_class[labels[i]] , fill=text_color,font=font)
    return image

In [41]:
model = model.cuda()

NameError: name 'model' is not defined

In [42]:
sahi_model = AutoDetectionModel.from_pretrained(
            model_type='torchvision',
            model=model,
            confidence_threshold=0.5,
            image_size=1024,
            device="cpu", # or "cuda:0"
            load_at_init=True,
            category_mapping ={"0": '_background_', "1": 'tire_marks', "2": 'trackout', "3": 'unclear'} )

NameError: name 'AutoDetectionModel' is not defined

In [43]:
idx=1 # till 25
# sahi_img_path = r"/home/sanjeev_kumar/work/usecase/trackouts_patch/Images/DJI_20220928175758_0125_2460_820_3484_1844.jpg"
# sahi_img_path = r"/home/sanjeev_kumar/work/data/updated_data/combined_v1_demo/images/DJI_20211123144931_0006_W.JPG"
sahi_img_path ="/home/sanjeev_kumar/work/trackouts/Images/DJI_20211123144914_0002_W.JPG"

# sahi_img_path= img_path_list[idx]
result = get_sliced_prediction(
    sahi_img_path,
    sahi_model,
    slice_height = 1024,
    slice_width = 1024,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
    postprocess_match_threshold=0.0  
)

NameError: name 'get_sliced_prediction' is not defined

In [44]:
idx="test_t"
# print(c?v2.imread(sahi_img_path).shape)

In [45]:
result_dict = result.to_coco_annotations()
print("result_dict : ",result_dict)
sahi_pred_list = extract_sahi_pred(result_dict,image_size = (1024, 1024))
print("sahi_pred_list : ",sahi_pred_list)

NameError: name 'result' is not defined

In [46]:
sahi_box_img = sahi_draw_bounding_box(sahi_img_path,sahi_pred_list)
save_dir =r"/home/sanjeev_kumar/work/usecase/trackouts_patch/pred_img/"
sahi_save_path = save_dir +"sahi_pred_"+str(idx)+".JPG"
sahi_box_img.save(sahi_save_path)

NameError: name 'sahi_pred_list' is not defined

In [47]:
for i,img_path in enumerate(img_path_list):
    result = get_sliced_prediction(
    sahi_img_path,
    sahi_model,
    slice_height = 1024,
    slice_width = 1024,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
    postprocess_match_threshold=0.01  
    )
    result_dict = result.to_coco_annotations()
    # print("result_dict : ",result_dict)
    sahi_pred_list = extract_sahi_pred(result_dict,image_size = (1024, 1024))

    sahi_box_img = sahi_draw_bounding_box(sahi_img_path,sahi_pred_list)
    save_dir =r"/home/sanjeev_kumar/work/usecase/trackouts_patch/pred_img/"
    sahi_save_path = save_dir +"sahi_b_pred_"+str(idx)+".JPG"
    sahi_box_img.save(sahi_save_path)

In [48]:
from PIL import Image
# Image.open(sahi_img_path)

In [49]:
object_prediction_list = result.object_prediction_list
object_prediction_list

NameError: name 'result' is not defined

In [50]:
result.export_visuals(export_dir="demo_data/")

NameError: name 'result' is not defined

In [51]:
Image("demo_data/prediction_visual.png")

TypeError: 'module' object is not callable

# sahi batch prediction

In [52]:
betch_result =predict(
    detection_model=sahi_model,
    source=UNSEEN_DATA_PATH,
    slice_height = 1024,
    slice_width = 1024,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
)

NameError: name 'predict' is not defined

# rough

In [53]:
sahi_model = get_sliced_prediction(
                img,
                detection_model,
                slice_height = 1024,
                slice_width = 1024,
                overlap_height_ratio = 0.2,
                overlap_width_ratio = 0.2,)

NameError: name 'get_sliced_prediction' is not defined

In [ ]:
device

In [ ]:
sahi_model = AutoDetectionModel.from_pretrained(
    model_type='torchvision',
    model=model,
    confidence_threshold=0.6,
    image_size=1024,
    device=device, # or "cuda:0"
    load_at_init=True,
)

In [ ]:
def get_sahi_model(model,img,img_size=(1024,1024),batch=False):
    if batch:
        sahi_model = AutoDetectionModel.from_pretrained(
                    model_type='torchvision',
                    model=model,
                    confidence_threshold=0.6,
                    image_size=1024,
                    device=device, # or "cuda:0"
                    load_at_init=True,
                    )
    else:
        sahi_model = get_sliced_prediction(
                img,
                detection_model,
                slice_height = 1024,
                slice_width = 1024,
                overlap_height_ratio = 0.2,
                overlap_width_ratio = 0.2,)
        